In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import Input
import matplotlib.pyplot as plt

# Define model
inputs = Input(shape=(5, 2))
lstm = tf.keras.layers.LSTM(4)
softmax = tf.keras.layers.Softmax()
output = softmax(lstm(inputs))
model = Model(inputs=inputs, outputs=output, name="my_lstm")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["acccuracy"])

# Model summary
model.summary()

In [ ]:
import pickle

# Load training/testing data
with open('../data/train_valid_test_data.pkl', 'rb') as f:
    X_train, y_train, X_valid, y_valid, X_test, y_test = pickle.load(f)

# Load tokenizer
with open('../data/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [40]:
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# We'll keep the model definition in a function
# The function takes an argument hp from which you can sample hyperparameters
from tensorflow.keras.optimizers import Adam, RMSprop

def build_model(hp):
    model = Sequential()
    model.add(Embedding(
        input_dim=1000,
        output_dim=hp.Int('output_dim', min_value=32, max_value=128, step=32),
        input_length=100)
    )
    model.add(LSTM(hp.Int('lstm_units', min_value=32, max_value=256, step=32)))
    model.add(Dense(1, activation='sigmoid'))

    # Add hyperparameters for the optimizer
    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop'])
    learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG', default=1e-3)

    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='random_search',
    project_name='sentiment_analysis'
)

INFO:tensorflow:Reloading Tuner from random_search/sentiment_analysis/tuner0.json


In [41]:
import keras_tuner
model = build_model(keras_tuner.HyperParameters())
tuner.search_space_summary()

Search space summary
Default search space size: 3
output_dim (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
lstm_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop'], 'ordered': False}


2023-06-04 19:24:41.241681: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-04 19:24:41.243107: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-04 19:24:41.244356: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
import numpy as np
print(np.all((X_train >= 0) & (X_train <= 998)))
print(np.all((X_valid >= 0) & (X_valid <= 999)))
print(set(y_train))
print(set(y_valid))

In [ ]:
print(type(X_train)) # should be <class 'numpy.ndarray'>
print(type(X_valid)) # should be <class 'numpy.ndarray'>
print(type(y_train)) # should be <class 'numpy.ndarray'> or <class 'list'>
print(type(y_valid)) # should be <class 'numpy.ndarray'> or <class 'list'>
print(X_train.shape) 
print(y_train.shape) 
print(X_valid.shape) 
print(y_valid.shape) 

In [ ]:
hist = model.fit(X_train, y_train, epochs=3, validation_data=(X_valid, y_valid))

In [ ]:
tuner.search(X_train, y_train, epochs=3, validation_data=(X_valid, y_valid))

In [ ]:
# Forward pass
X = tf.random.normal((10, 5, 2))
pred = model(X)
print(pred)

In [ ]:
for layer in model.layers:
    weights = layer.get_weights()  # list of numpy arrays
    print(weights)
    if len(weights)>0:
        for w in weights:
            print(w.shape)
    print("=======")

In [ ]:
for layer in model.layers:
    weights = layer.get_weights()  # list of numpy arrays
    for weight in weights:
        plt.hist(weight.flatten())
        plt.show()

In [ ]:
import tensorflow as tf

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
x_sample = tf.random.normal((3, 5, 2))
y_sample = [1, 2, 2]

# suppose x_sample and y_sample are your input features and labels
y_pred = model.predict(x_sample)

loss = loss_fn(y_sample, y_pred)
print('Loss:', loss.numpy())

In [ ]:
import numpy as np
import tensorflow as tf

num_samples = 100
seq_length = 10
num_features = 2
num_classes = 5
train_ratio = 0.8
num_lstm_cells = 10
train_split = int(train_ratio*num_samples)

# Generate sequential data where each value depends on the previous one
X = np.empty((num_samples, seq_length, num_features))
for i in range(num_samples):
    for j in range(seq_length):
        for k in range(num_features):
            if j > 0:
                X[i, j, k] = X[i, j - 1, k] * 0.9 + np.random.normal(0, 0.1)
            else:
                X[i, j, k] = np.random.normal(0, 0.1)

# Generate labels that are a function of the last value in each sequence
y = np.sum(X[:, -1, :], axis=1)
y = np.round(y).astype(int)  # convert to integers for classification
y = np.clip(y, 0, num_classes - 1)  # clip to the range 0 to num_classes - 1
y = tf.keras.utils.to_categorical(y, num_classes=num_classes)

X_train = X[:train_split]
y_train = y[:train_split]

X_test = X[train_split:]
y_test = y[train_split:]

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(num_lstm_cells, input_shape=(seq_length, num_features)),  # LSTM layer
    tf.keras.layers.Dense(num_classes, activation='softmax')  # Output layer for multi-class classification
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
hist = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=tensorboard_callback)

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation accuracy values
plt.subplot(1, 2, 2)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.tight_layout()
plt.show()